1. Use shell script to unzip all the data
2. Use awk to further clean the data
3. Drop the data into local sql

In [1]:
import pandas as pd
import numpy as np
import sqlite3 as sql

In [2]:
musicbox_conn = sql.connect("musicbox.sqlite")
musicbox_conn.text_factory = str

In [3]:
schema_down = ['uid','device','song_id','song_name','singer','paid_flag','date']

In [4]:
df_down = pd.read_table("./ready/download/all_down_7.log.fn", header=None, names=schema_down, parse_dates=['date'])

In [6]:
df_down.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7737325 entries, 0 to 7737324
Data columns (total 7 columns):
uid          int64
device       object
song_id      float64
song_name    object
singer       object
paid_flag    int64
date         datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(2), object(3)
memory usage: 413.2+ MB


In [9]:
df_down.head()

,uid,device,song_id,song_name,singer,paid_flag,date
0,168019810,ar,442554.0,小酒窝,林俊杰&蔡卓妍,0,2017-03-30
1,168019810,ar,6334611.0,社会摇,萧全,0,2017-03-30
2,168019810,ar,9867382.0,台阶,武艺,0,2017-03-30
3,168019810,ar,6660691.0,一次就好-(电影《夏洛特烦恼》暖水曲),杨宗纬,0,2017-03-30
4,168019810,ar,157606.0,一路上有你,张学友,0,2017-03-30


In [9]:
df_down.song_name = df_down.song_name.str.strip()
df_down.singer = df_down.singer.str.strip()

In [14]:
%%time
df_down.to_sql('Down', musicbox_conn, if_exists='replace', index=False)

CPU times: user 50.8 s, sys: 4.51 s, total: 55.3 s
Wall time: 57.6 s


In [21]:
pd.read_sql("SELECT * FROM Down LIMIT 5", musicbox_conn)

,uid,device,song_id,song_name,singer,paid_flag,date
0,168019810,ar,442554.0,小酒窝,林俊杰&蔡卓妍,0,2017-03-30 00:00:00
1,168019810,ar,6334611.0,社会摇,萧全,0,2017-03-30 00:00:00
2,168019810,ar,9867382.0,台阶,武艺,0,2017-03-30 00:00:00
3,168019810,ar,6660691.0,一次就好-(电影《夏洛特烦恼》暖水曲),杨宗纬,0,2017-03-30 00:00:00
4,168019810,ar,157606.0,一路上有你,张学友,0,2017-03-30 00:00:00


In [16]:
%%time
pd.read_sql("SELECT COUNT(uid) FROM Down", musicbox_conn)

CPU times: user 1.3 s, sys: 425 ms, total: 1.73 s
Wall time: 2.29 s


,COUNT(uid)
0,7737325


In [26]:
%%time
! wc -l ./ready/download/all_down_7.log.fn

 7737325 ./ready/download/all_down_7.log.fn
CPU times: user 11.1 ms, sys: 37.3 ms, total: 48.4 ms
Wall time: 878 ms


## play data

In [3]:
schema_play = ['uid','device','song_id','song_type','song_name','singer','play_time','song_length',
               'paid_flag','date']

data_types = {'uid': np.int64,
              'device': str,
              'song_id': np.float64,
              'song_type': np.float64,
              'song_name': str,
              'singer': str,
              'play_time': np.float64,
              'song_length': np.float64,
              'paid_flag': np.int64}

In [23]:
play_reader = pd.read_table("./ready/play/all_play_10.log.fn", header=None, names=schema_play, 
                            dtype=data_types, na_values=[' ', 'nan '], parse_dates=['date'],
                            chunksize=500000, iterator=True)

In [24]:
%%time
play_chunk_first = play_reader.next()
play_chunk_first.device = play_chunk_first.device.str.strip()
play_chunk_first.song_name = play_chunk_first.song_name.str.strip()
play_chunk_first.singer = play_chunk_first.singer.str.strip()

play_chunk_first.to_sql('Play', musicbox_conn, if_exists='replace', index=False)
for play_chunk in play_reader:
    try:
        play_chunk.device = play_chunk.device.str.strip()
        play_chunk.song_name = play_chunk.song_name.str.strip()
        play_chunk.singer = play_chunk.singer.str.strip()
        play_chunk.to_sql('Play', musicbox_conn, if_exists='append', index=False)
    except:
        print "Oops!"
        break

CPU times: user 28min 37s, sys: 1min 50s, total: 30min 27s
Wall time: 31min 11s


In [25]:
pd.read_sql("SELECT * FROM sqlite_master", musicbox_conn)

,type,name,tbl_name,rootpage,sql
0,table,Down,Down,2,"CREATE TABLE ""Down"" (\n""uid"" INTEGER,\n ""devi..."
1,table,Play,Play,130839,"CREATE TABLE ""Play"" (\n""uid"" INTEGER,\n ""devi..."


In [26]:
%%time
pd.read_sql("SELECT COUNT(*) FROM Play", musicbox_conn)

CPU times: user 1.88 s, sys: 9.53 s, total: 11.4 s
Wall time: 34.4 s


,COUNT(*)
0,164487071


In [27]:
%%time
! wc -l ./ready//play/all_play_10.log.fn

 164487071 ./ready//play/all_play_10.log.fn
CPU times: user 215 ms, sys: 97.2 ms, total: 312 ms
Wall time: 18.8 s


In [8]:
tmp = pd.read_table("./play_sample/20170401_1_play.log", header=None, names=schema_play, 
                    dtype=data_types, na_values=[' ', 'nan '], parse_dates=['date'])

ValueError: Integer column has NA values in column 0

In [6]:
(tmp.song_name == '').mean()

0.0